<a href="https://colab.research.google.com/github/SofiaCR2/Python-basico-intermedio/blob/main/ch26_SlqiteDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de datos SQLite
- https://www.sqlite.org/
- Basado en C, una de las bases de datos integradas más utilizadas (configuración cero)

## Conceptos básicos de SQL
- Lenguaje de consulta estructurado
- lenguaje insensible a mayúsculas y minúsculas; generalmente escrito en mayúsculas
- Permita que usted o el programa utilicen bases de datos basadas en SQL como SQLite, MySQL, MSSQL, PostgreSQL, etc.
- declaraciones básicas más importantes para aprender: CRUD
- C: crear (base de datos, tabla, crear e insertar registros)
- R: recuperar/leer datos
-U: actualizar datos
- D: eliminar datos
- http://www.w3schools.com/sql/default.asp

## navegador sqlite
- Explorador de sqlite db basado en GUI
- facilita la visualización de datos y el aprendizaje de SQL
- http://sqlitebrowser.org/

## módulo sqlite3
- python3 proporciona la biblioteca sqlite3 para trabajar con la base de datos sqlite
- https://docs.python.org/3/library/sqlite3.html
- SQLite admite de forma nativa los siguientes tipos: NULL, INTEGER, REAL, TEXT, BLOB

|SQLite type|Python type|
| ---|---|
| NULL | None |
| INTEGER | int |
| REAL | float |
| TEXT | str |
| BLOB | bytes |

## en el ejemplo de base de datos de memoria

In [1]:
import sqlite3
# connect to the memory database
con = sqlite3.connect(":memory:")

# create a table
con.execute("create table person(fname, lname)")

In [6]:
# fill the table with data
persons = [('Hugo', 'Boss'), ('Calvin', 'Klien')]
con.executemany("insert into person(fname, lname) values (?, ?)",
                persons)

In [7]:
# print the table contents
for row in con.execute("select rowid, fname, lname from person"):
    print(row)

(2, 'Calvin', 'Klien')
(3, 'Hugo', 'Boss')
(4, 'Calvin', 'Klien')


In [8]:
print("I just deleted", con.execute("delete from person where rowid=1").rowcount,
      "rows")

I just deleted 0 rows


## ejemplo de archivo db
### crear base de datos, crear tabla e insertar datos en la tabla

In [9]:
import sqlite3
# create connection
conn = sqlite3.connect('example.db')
# create cursor object
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS students (
                firstName text,
                lastName text,
                test1 real,
                test2 real,
                average real,
                grade text
                )
            """)

In [10]:
query = """ INSERT INTO students (firstName, lastName,
            test1, test2) values (?, ?, ?, ?)
        """
cur.execute(query, ('John', 'Smith', 99, 95.5))

In [11]:
cur.execute(query, ('Michael', 'Jordan', 50, 65))

In [12]:
# save/commit the changes to the db
conn.commit()
# close the database if done
conn.close()

### abrir base de datos, leer y actualizar tabla

In [13]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

In [14]:
cur.execute('SELECT * FROM students where rowid = 1')
row = cur.fetchone() # returns one row as tuple if rowid with value 1 exists
print(row)

('John', 'Smith', 99.0, 95.5, None, None)


In [15]:
for col in row:
    print(col)

John
Smith
99.0
95.5
None
None


In [17]:
cur.execute('SELECT rowid, * FROM students')
rows = cur.fetchall()
print(rows)

[(1, 'John', 'Smith', 99.0, 95.5, None, None), (2, 'Michael', 'Jordan', 50.0, 65.0, None, None)]


In [18]:
for row in rows:
    print(row)

(1, 'John', 'Smith', 99.0, 95.5, None, None)
(2, 'Michael', 'Jordan', 50.0, 65.0, None, None)


<strong>update tabla</strong>

In [19]:
for row in rows:
    avg = (row[3] + row[4])/2
    # grade = ?
    cur.execute('update students set average=? where rowid=?', (avg, row[0]))

In [20]:
cur.execute('select * from students')
print(cur.fetchall())

[('John', 'Smith', 99.0, 95.5, 97.25, None), ('Michael', 'Jordan', 50.0, 65.0, 57.5, None)]


In [21]:
# commit changes and close connection
conn.commit()
conn.close()

## Vulnerabilidad de inyección SQL
- cómo no escribir consultas sql en programas

In [22]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS users (
                username text unique,
                password text
                )
            """)

In [23]:
# Prompt user to create account
username = input('Enter your username: ')
password = input('Pick a password: ')

Enter your username: sofia
Pick a password: 1234


In [24]:
# bad passwords
# insecure way to create sql statements
sqlinsert = "insert into users (username, password) values ('{0}', '{1}')".format(username, password)
print(sqlinsert)
cur.execute(sqlinsert)


insert into users (username, password) values ('sofia', '1234')


In [25]:
# check database
conn.commit()
for row in cur.execute('select * from users'):
    print(row)

('sofia', '1234')


### ¿Qué hay de malo en los códigos anteriores?

### autenticación de usuarios y ataque de inyección SQL

In [26]:
# Prompt user to create account
def insecureAuthentication():
    username = input('Enter your username: ')
    password = input('Pick a password: ')
    sqlSelect = "select * from users where username = '{0}' \
                    and password = '{1}'".format(username, password)
    cur.execute(sqlSelect)
    row = cur.fetchone()
    if row:
        print('Welcome {}, this is your kingdom!'.format(row[0]))
    else:
        print('Wrong credentials. Try Again!')


In [28]:
insecureAuthentication()

Enter your username: sofia
Pick a password: 1234
Welcome sofia, this is your kingdom!


In [30]:
# sql injection; authenticate without using password
insecureAuthentication()

Enter your username: sofia
Pick a password: 1234
Welcome sofia, this is your kingdom!


## forma segura de almacenar la contraseña
- https://docs.python.org/3/library/hashlib.html


In [31]:
import uuid
import hashlib, binascii

def createSecurePassword(password, salt=None, round=100000):
    if not salt:
        salt = uuid.uuid4().hex

    """
    for i in range(round):
        password = password+salt
        password = hashlib.sha256(password.encode('utf-8')).hexdigest()
    """
    # hashlib.pbkdf2_hmac(hash_name, password, salt, iterations, dklen=None)
    dk = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'),
                        salt.encode('utf-8'), round)
    password = binascii.hexlify(dk)
    return "%s:%s"%(password, salt)

In [32]:
def secureRegistration():
    # Prompt user to create account
    username = input('Enter your username: ')
    password = input('Enter your password: ')
    secPass = createSecurePassword(password)
    insert = 'insert into users (username, password) values (?, ?)'
    cur.execute(insert, (username, secPass))


In [35]:
# register a user
secureRegistration()

Enter your username: lola
Enter your password: joya


In [36]:
# check data
for row in cur.execute('select * from users'):
    print(row)

('sofia', '1234')
('lola', "b'cd32bad31c64e611c50c6232d2e54c66ecc55c86d60dc2653e21eadf15727234':ccb0541124604dae994733ea2402482a")


In [37]:
conn.commit()

In [38]:
def secureAuthentication():
    username = input('Enter your username: ')
    password = input('Enter your password: ')
    # use parameterized query
    sqlSelect = 'select password from users where username = ?'
    cur.execute(sqlSelect, (username,))
    row = cur.fetchone()
    if row:
        # username exists
        # check password hashes
        hashpass = row[0]
        hashedPass = hashpass[:hashpass.find(':')]
        salt = hashpass[hashpass.find(':')+1:]
        secPass = createSecurePassword(password, salt)
        if hashpass == secPass:
            print('Welcome to your kingdom, {}'.format(username))
        else:
            print('Wrong credentials. Try Again!')
    else:
        print('Wrong credentials. Try Again!')

In [41]:
secureAuthentication()

Enter your username: sofia
Enter your password: 1234
Wrong credentials. Try Again!


In [42]:
# try the same SQL injection
secureAuthentication()

Enter your username: lola
Enter your password: joya
Welcome to your kingdom, lola


In [43]:
conn.commit()
conn.close()